### Встроенная реализация Pyro
Эксперимент с использованием встроенной в Pyro реализации фильтра Калмана
с целью определить, можно ли использовать её для дальнейшей работы.

Был обучен фильтр Калмана (матрицы ковариаций шума наблюдения и шума процесса) для модели движения объекта в одномерном пространстве.
Замечено, что:
1. Способ задания динамической модели во встроенной реализации фильтра Калмана переусложнён.
2. Встроенная реализация не позволяет обучать матрицы эволюции процесса и наблюдения, что ограничивает её применение.

![Результат фильтрации искуственно созданного сигнала](img/kalman-pyro-contrib.png)

Вывод: целесобразно создание собственной реализации фильтра Калмана.

### Собственная реализация
Эксперименты с использованием собственной реализации фильтра Калмана.

#### Идентификация динамической модели для зашумлённой синусоиды
Были созданы искусственные данные в виде 100 отсчётов синусоиды с добавленным гауссовым шумом.
Производились попытки выучить матрицы эволюции процесса, наблюдения, ковариации шума наблюдения, ковариации шума процесса.

Полученная модель оказалась корректно работающей и способной выдать плавную синусоиду в отдельных ситуациях,
однако оптимизационная задача, задаваемая ею оказалась сложной и
для получения приемлемых результатов даже для относительно небольшого уровня шума
потребовала значительного тюнинга процедуры оптимизации.

Результаты обучения фильтра такими методами, как AdaDelta и Adam оказались непредсказуемыми (зависящами от случайной инициализации?)
и часто неудовлетворительными.

Результаты фильтрации при неоптимальном решении оптимизационной задачи выглядели как недостаточно сильно отфильтрованные входные данные.
Пример результатов приведён на изображении ниже.
![Результат обучения фильтра Калмана с помощью метода SdLBFGS](img/kalman-sine-grad.png)

Вывод:
1. идея об обучении динамической модели имеет смысл
2. для развития идеи требуются более эффективные методы математической оптимизации,
   либо же способ штрафовать фильтр за неиспользование предсказаний из динамической модели

#### Использование методов второго порядка для обучения фильтра Калмана
Так как оптимизационная задача, возникшая в предыдущем эксперименте оказалась слишком сложной для методов, встроенных в PyTorch,
было решено воспользоваться каким-либо методом второго порядка.

Для PyTorch была найдена реализация модифицированного метода [SdLBFGS](https://www.groundai.com/project/implementation-of-stochastic-quasi-newtons-method-in-pytorch/), представляющего собой модификацию метода LBFGS, приспособленную для стохастической оптимизации.

Данный метод показал очень быструю сходимость, значительно вревосходя методы AdaDelta и Adam.
Кроме того, решение, найденное с помощью этого метода оказалось значительно лучше, чем у методов второго порядка.
Результат обучения фильтра Калмана этим методом приведён на изображении ниже.

![Результат обучения фильтра Калмана с помощью метода SdLBFGS](img/kalman-sine-sdlbfgs.png)

Тем не менее, несмотря на заметные улучшения, конечный результат всё ещё непредсказуем
и далеко не каждая попытка обучения приводит к результату, приведённому на изображении выше.

Вывод:
требуется найти способ штрафовать фильтр за неиспользование предсказаний из динамической модели

#### Обучение фильтра Калмана на основе постоянно поступающих данных
Эксперимент, в целом аналогичный приведённому выше, однако обучение происходило по одному большому синусоидальному сигналу.
На каждом шаге из единого сигнала выбирались следующие 100 отсчётов и обучение происходило по ним.

Изначально существовали опасения, что в начале обучения фильтр может быть нестабильным и передевать на следующий шаг бредовые значения состояния системы, что приводило бы к ещё большей нестабильности на следующих шагах,
однако, на практике, такого являения не наблюдалось.

Тем не менее, данный эксперимент оказался неудачным: методы первого порядка продолжили давать неудовлетворительный результат, а после обучения с помощью SdLBFGS фильтр начинал копировать входные данные.

#### Модификация функции потерь для упрощения оптимизационной задачи обучения фильтра Калмана
Величина, пропорциональная амплитуде высокочастотной части предсказания фильтра
использовалась в качестве добавочного терма для функции потерь.
Интуитивно, данная добавка стимулировала более гладкие предсказания, которые невозможно сделать копируя шумные входные данные.
Высокочастотная часть выделялась с помощью простого КИХ фильтра.

Во время обучения множитель данного терма постепенно уменьшалась до нуля, так что в итоге оптимизировалась исходная функция.

Данная модификация оказалась эффективной.
После её применения процедура обучения начала более стабильно выдавать сглаживающий фильтр
как для фиксированных данных, так и для постоянно поступающих.

Ниже приведён пример фильтрации постоянно поступающего сигнала.

![Результат обучения фильтра Калмана со сглаживающим термом на постоянно поступающих данных](img/kalman-sine-online.png)